# This notebook walk through a text processing, in an attemp to mimic hand classification process


# load libraries

In [95]:
import numpy as np
import pandas as pd
import re
import os
import nltk
from nltk.stem.porter import PorterStemmer
#nltk.download('stopwords')
from nltk.corpus import stopwords
ps = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=4000) # 4000 most freqeunt words only
from collections import Counter

# read data

In [214]:
rawdata = pd.read_csv('/home/amir/Downloads/rawdata_1.txt',sep='\t',encoding = "ISO-8859-1")
rawdata.head()

,Id,CompanyName,ShortDescription,Description,YearFounded,Type,Status,SectorID,SubSectorId,ProductStageID,...,LookingForInvestorsSum,LookingForNewCustomers,SubtrendId,TrendId,INV_FinancialInformation.MaximumInvestment,INV_FinancialInformation.MinimumInvestment,IsFoodTech,IsActive,AreaID,RaisedUpdated
0,1,Flying SpArk,Insect Larvae-extracted Proteins,Flying SpArk produces a high-quality protein p...,2014.0,2,11.0,1.0,8.0,5.0,...,$3.5m,False,25.0,4.0,NaN,NaN,1.0,1.0,4.0,1020000.0
1,2,ClariFruit,Quality Detection for Produce,ClariFruit is an innovative start-up introduci...,2016.0,2,7.0,1.0,NaN,6.0,...,NaN,True,2.0,1.0,NaN,NaN,1.0,1.0,7.0,NaN
2,3,Innovopro,Protein Isolation Technology,InnovoPro has developed a technology to extrac...,2013.0,2,7.0,1.0,19.0,6.0,...,NaN,False,28.0,4.0,NaN,NaN,1.0,1.0,7.0,0.0
3,4,bitemojo,Self-guided Culinary Experiences,bitemojo offers self-guided tours designed to ...,2016.0,2,8.0,6.0,NaN,6.0,...,NaN,False,36.0,6.0,NaN,NaN,NaN,1.0,2.0,578500.0
4,5,ChickP,Chickpea-based Concentrate and Isolate,ChickP has developed a novel production proces...,2016.0,2,6.0,1.0,NaN,6.0,...,NaN,False,28.0,4.0,NaN,NaN,1.0,1.0,7.0,500000.0


In [97]:
rawdata.shape

(1052, 40)

# columns

In [98]:
rawdata.columns

Index(['Id', 'CompanyName', 'ShortDescription', 'Description', 'YearFounded',
       'Type', 'Status', 'SectorID', 'SubSectorId', 'ProductStageID',
       'SizeCompany', 'Raised', 'LogoPictureURL', 'GeoMarkets', 'IsAcquired',
       'HomePage', 'Linkedin', 'Facebook', 'GooglePlus', 'Twitter',
       'HUB_Duration', 'HUB_Batches_Years', 'HUB_BatchSize',
       'INV_ManagedAssets', 'INV_StagesID', 'INV_TypeId',
       'INV_HUB_IndustryPreferencesID', 'ModificationDate', 'CreationDate',
       'LookingForInvestors', 'LookingForInvestorsSum',
       'LookingForNewCustomers', 'SubtrendId', 'TrendId',
       'INV_FinancialInformation.MaximumInvestment',
       'INV_FinancialInformation.MinimumInvestment', 'IsFoodTech', 'IsActive',
       'AreaID', 'RaisedUpdated'],
      dtype='object')

filter nulls for trend id and descriptions

# filter nulls for "description" and "TrendID"

In [224]:
dataset=rawdata[(rawdata.Description.notnull())&(rawdata.TrendId.notnull())]

dataset.index = range(0,dataset.shape[0])
dataset.shape

(389, 40)

pre process text

# clean text: remove stop words, lower case, stem, leave only english letter

In [153]:

corpus = []

for i in range(0,dataset.shape[0]):
        #remove everyting but letters (numbers symbols)
        resume=re.sub('[^a-zA-z]',' ',dataset.loc[i,'Description'])
        resume=resume.replace("_","")
        #lower case
        resume=resume.lower()
        #stop words (and this) and stemming
        resume=resume.split()
        resume=[ps.stem(word) for word in resume if not word in stopwords.words('english')]
        resume=' '.join(resume)
        corpus.append(resume)
        

In [154]:
dataset.loc[0,'Description']

'Flying SpArk produces a high-quality protein powder made from fruit fly larvae_ offering a healthy and sustainable alternative to conventional animal protein.Fruit flies are rich in iron_ calcium_ and magnesium. Their extremely short life cycle and rapid growth make them one of the most affordable sources of animal protein available today.Insect protein is cholesterol free and has an exceptional ratio of omega-3 to omega-6 fatty acids and a similar profile of essential amino acids. It is also free of antibiotics_ pesticides_ and hormones. In addition_ insects have a minimal ecological footprint_ utilizing virtually no water or land and contributing neither to waste nor to greenhouse gas emissions.Flying SpArk is part of The Kitchen FoodTech Hub Incubator.'

In [106]:
corpus[0]

'fli spark produc high qualiti protein powder made fruit fli larva offer healthi sustain altern convent anim protein fruit fli rich iron calcium magnesium extrem short life cycl rapid growth make one afford sourc anim protein avail today insect protein cholesterol free except ratio omega omega fatti acid similar profil essenti amino acid also free antibiot pesticid hormon addit insect minim ecolog footprint util virtual water land contribut neither wast greenhous ga emiss fli spark part kitchen foodtech hub incub'

# TFIDF VECTORS

In [210]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=45000,ngram_range=(1,2),max_df=0.2,min_df=2)
features = tfidf.fit_transform(corpus)
output = pd.DataFrame(
        features.todense(),
        columns=tfidf.get_feature_names()
        )

output=output.values
output_pd=pd.DataFrame(output,columns=tfidf.get_feature_names())

In [211]:
output_pd.iloc[30:40,:]

,abil,abl,abl provid,absorb,academ,acceler,access,accord,accord custom,account,...,xsens,year,year experi,year expertis,year round,yet,yet simpl,yield,yogurt,zero
30,0.0,0.0,0.0,0.0,0.0,0.000000,0.092232,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
31,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
32,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
33,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.164254
34,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
35,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
36,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
37,0.0,0.0,0.0,0.0,0.0,0.116718,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
38,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
39,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


# train test stratified split

In [212]:
dataset["TrendId"] = dataset["TrendId"].replace({9:7,10:8})

X_train, X_test, y_train, y_test = train_test_split(output_pd, dataset['TrendId'], test_size=0.5, random_state=42,stratify=dataset['TrendId'])

X_train.index=(range(0,X_train.shape[0]))
X_test.index=(range(0,X_test.shape[0]))
y_train.index=(range(0,y_train.shape[0]))
y_test.index=(range(0,y_test.shape[0]))

/home/amir/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [202]:
y_train.value_counts(normalize=True)

1.0    0.283505
2.0    0.201031
6.0    0.149485
7.0    0.144330
3.0    0.092784
5.0    0.046392
8.0    0.041237
4.0    0.041237
Name: TrendId, dtype: float64

In [203]:
y_test.value_counts(normalize=True)

1.0    0.287179
2.0    0.205128
7.0    0.148718
6.0    0.143590
3.0    0.087179
5.0    0.046154
4.0    0.041026
8.0    0.041026
Name: TrendId, dtype: float64

# calculate 8 mean vectors for each trend out of our train vectors

In [213]:
mean_vectors=X_train.groupby(y_train, as_index=False).mean()
mean_vectors

,abil,abl,abl provid,absorb,academ,acceler,access,accord,accord custom,account,...,xsens,year,year experi,year expertis,year round,yet,yet simpl,yield,yogurt,zero
0,0.004175,0.003037,0.002519,0.000000,0.001158,0.008008,0.004421,0.000000,0.000000,0.000000,...,0.006328,0.007427,0.004719,0.0,0.000000,0.004370,0.004857,0.005179,0.000000,0.000000
1,0.005006,0.000000,0.000000,0.003052,0.000000,0.000000,0.011113,0.008705,0.000000,0.000000,...,0.000000,0.003423,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.007611
2,0.000000,0.013448,0.007547,0.000000,0.009497,0.000000,0.000000,0.016070,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.008999,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010811,0.000000,0.0,0.000000,0.016810,0.000000,0.031763,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011810,0.015745,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.011965,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005576,0.011781,0.007853,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.005204,0.005784,0.000000,0.000000,0.000000
6,0.006858,0.000000,0.000000,0.000000,0.005481,0.000000,0.000000,0.003927,0.000000,0.003717,...,0.000000,0.008102,0.004736,0.0,0.006331,0.003796,0.000000,0.014117,0.000000,0.011310
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# calculate similarity matrix for tet vectors

In [216]:
from sklearn.metrics.pairwise import cosine_similarity
trend_mapping=cosine_similarity(X_test,mean_vectors)


In [217]:
trend_mapping_pd = pd.DataFrame(trend_mapping)
trend_mapping_pd.head()

,0,1,2,3,4,5,6,7
0,0.172161,0.054943,0.021193,0.031967,0.071892,0.145680,0.031220,0.071103
1,0.131632,0.100997,0.078339,0.091892,0.085621,0.044893,0.102114,0.051809
2,0.021833,0.064372,0.019015,0.020477,0.023042,0.074232,0.011484,0.013264
3,0.024627,0.052461,0.034935,0.014147,0.000000,0.013444,0.039496,0.011573
4,0.090223,0.131285,0.102375,0.091358,0.039943,0.047861,0.170700,0.030838


# predict using index of most similar vector

In [219]:
trend_mapping_pd["predicted_trend"] = trend_mapping_pd.iloc[:,0:8].idxmax(axis=1)+1
trend_mapping_pd["predicted_trend"].value_counts(normalize=True)

1    0.292308
2    0.179487
6    0.174359
7    0.143590
5    0.076923
4    0.051282
3    0.046154
8    0.035897
Name: predicted_trend, dtype: float64

# confusion matrix

In [220]:
from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, trend_mapping_pd['predicted_trend'])

array([[36,  4,  2,  1,  2,  5,  5,  1],
       [ 3, 25,  0,  1,  1,  8,  2,  0],
       [ 5,  0,  7,  0,  2,  2,  1,  0],
       [ 0,  1,  0,  7,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  9,  0,  0,  0],
       [ 4,  2,  0,  1,  0, 19,  0,  2],
       [ 6,  3,  0,  0,  0,  0, 20,  0],
       [ 3,  0,  0,  0,  1,  0,  0,  4]])

# confusion reprot

In [221]:
from sklearn.metrics import classification_report
print(classification_report(y_test, trend_mapping_pd['predicted_trend']))

              precision    recall  f1-score   support

         1.0       0.63      0.64      0.64        56
         2.0       0.71      0.62      0.67        40
         3.0       0.78      0.41      0.54        17
         4.0       0.70      0.88      0.78         8
         5.0       0.60      1.00      0.75         9
         6.0       0.56      0.68      0.61        28
         7.0       0.71      0.69      0.70        29
         8.0       0.57      0.50      0.53         8

   micro avg       0.65      0.65      0.65       195
   macro avg       0.66      0.68      0.65       195
weighted avg       0.66      0.65      0.65       195



# total accuracy

In [222]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test, trend_mapping_pd['predicted_trend'])

print("final accuracy of predictiong on test: {}".format(acc))

final accuracy of predictiong on test: 0.6512820512820513
